In [1]:
!pip install -q grobid-client langchain openai faiss-cpu PyPDF2
!pip install -U google-cloud-aiplatform --user

In [2]:
PROJECT_ID = "esoteric-stream-399606"  # @param {type:"string"}
LOCATION = "us-central1" # @param {type:"string"}

In [3]:
from vertexai.preview.language_models import TextGenerationModel,\
                                            ChatModel,\
                                            InputOutputTextPair,\
                                            TextEmbeddingModel

from IPython.display import display, Markdown
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns

from google.colab import auth as google_auth
google_auth.authenticate_user()

import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [4]:
chat_model = ChatModel.from_pretrained("chat-bison@001")
print(type(chat_model))

chat = chat_model.start_chat()
print(type(chat))

print(chat.send_message("""
My name is Larry.
"""))

<class 'vertexai.preview.language_models._PreviewChatModel'>
<class 'vertexai.preview.language_models._PreviewChatSession'>
MultiCandidateTextGenerationResponse(text='Hi Larry, how can I help you today?', _prediction_response=Prediction(predictions=[{'groundingMetadata': [None], 'safetyAttributes': [{'blocked': False, 'categories': [], 'scores': []}], 'citationMetadata': [{'citations': []}], 'candidates': [{'content': 'Hi Larry, how can I help you today?', 'author': '1'}]}], deployed_model_id='', model_version_id='', model_resource_name='', explanations=None), is_blocked=False, safety_attributes={}, candidates=[Hi Larry, how can I help you today?])


In [12]:
for m in chat.message_history :
  print(m.author, ':', m.content)
  print("")

user : 
My name is Larry.

bot : Hi Larry, how can I help you today?
user : 
What's my name again?

bot : Your name is Larry.
user : 
What's my name again?

bot : Your name is Larry.


In [13]:
chat = chat_model.start_chat(
    context="My name is Ned. You are my personal assistant. My favorite movies are Lord of the Rings and Hobbit.",
    examples=[
        InputOutputTextPair(
            input_text="Who do you work for?",
            output_text="I work for Ned.",
        ),
        InputOutputTextPair(
            input_text="What do I like?",
            output_text="Ned likes watching movies.",
        ),
    ],
    temperature=0.3,
    max_output_tokens=1024,
    top_p=0.8,
    top_k=1
)
print(chat.send_message("Are my favorite movies based on a book series?").text)

Yes, your favorite movies are based on a book series.


In [5]:
!pip install transformers
!pip install sqlalchemy
!pip install mysql-connector-python
import sys
!{sys.executable} -m pip install cloud-sql-python-connector["pg8000"] SQLAlchemy==2.0.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 102.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 39.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.21.12 whic

In [6]:
import pg8000
from google.cloud.sql.connector import Connector, IPTypes
import sqlalchemy
from transformers import AutoModel, AutoTokenizer

def get_KoSimCSE():
    model = AutoModel.from_pretrained('BM-K/KoSimCSE-roberta-multitask')
    tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-roberta-multitask')

    print(type(model))
    print(type(tokenizer))

    return model, tokenizer

model, tokenizer = get_KoSimCSE()

<class 'transformers.models.roberta.modeling_roberta.RobertaModel'>
<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>


In [7]:
instance_connection_name = "esoteric-stream-399606:asia-northeast3:wjdfoek3"
db_user = "postgres"
db_pass = "pgvectorwjdfo"
db_name = "pgvector"

# initialize Cloud SQL Python Connector object
connector = Connector()

def getconn() -> pg8000.dbapi.Connection:
    conn: pg8000.dbapi.Connection = connector.connect(
        instance_connection_name,
        "pg8000",
        user=db_user,
        password=db_pass,
        db=db_name,
        ip_type=IPTypes.PUBLIC,
    )
    return conn

pool = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=getconn,
)

with pool.connect() as db_conn:
  db_conn.execute(
      sqlalchemy.text(
          "CREATE EXTENSION IF NOT EXISTS vector with schema public"
      )
  )
  db_conn.commit()

# pool.connect()하고 한 transaction만 수행 가능함.
# transaction 하나 수행하고 commit, close 하고 다시 connect 하기

In [54]:
with pool.connect() as db_conn:
  db_conn.execute(
    sqlalchemy.text(
      "CREATE EXTENSION IF NOT EXISTS pg_trgm with schema public"
    )
  )
  db_conn.commit()

#example
with pool.connect() as db_conn:

  db_conn.execute(sqlalchemy.text("DROP TABLE IF EXISTS empv4"))

  db_conn.execute(
    sqlalchemy.text(
      """
      create table empv4 (
        ename varchar,
        empno int primary key,
        org_text varchar(4096),
        evec vector(768)
      )
      """
    )
  )

  db_conn.commit()

In [55]:
sampel_text = [
    "안녕하세요",
    "반갑습니다",
    "안녕하세요 반갑습니다",
    "안녕하세요 반갑네요",
    "서울에 갑니다",
    "인천에 갑니다",
    "부산에 갑니다",
    "학교에 갑니다",
    "사무실에서 나옵니다",
    "사무실이 먼가요",
    "지하철을 탑니다",
    "버스를 탑니다",
]

In [56]:
with pool.connect() as db_conn:

    empno = 5555
    for text in sampel_text:

        inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

        embeddings, _ = model(**inputs, return_dict=False)
        embedding_arr = embeddings[0][0].detach().numpy()
        embedding_str = ",".join(str(x) for x in embedding_arr)
        embedding_str = "["+embedding_str+"]"

        insert_stmt = sqlalchemy.text(
            "INSERT INTO empv4 VALUES (:ename, :empno, :org_text, :evec)"
        )

        empno += 1
        ename = "emp"+str(empno)

        db_conn.execute(
            insert_stmt, parameters={"ename": ename, "empno": empno, "org_text": text, "evec": embedding_str}
        )

    db_conn.commit()

In [60]:
with pool.connect() as db_conn:
    result = db_conn.execute(
        sqlalchemy.text(
            "SELECT * FROM empv4"
        ),
    ).fetchall()

for row in result:
    print(row[2], row[3])

안녕하세요 [-0.74159324,-0.3165547,-0.0849291,-0.06452261,0.7979546,-0.24241632,-0.100766234,0.004376532,0.043259643,-0.12593642,-0.57392585,0.18496929,-0.8081505,0.282821,0.29552227,0.57174236,0.07761382,0.16505572,-0.24089228,0.9892372,0.069305696,0.04086633,0.00063061685,0.37803504,0.40414828,1.0443234,0.6425033,0.04059862,0.7050597,-0.4790375,-0.28874987,-0.1231131,-0.0073079504,-0.4314333,0.9848504,-0.21966909,-0.03250195,0.245562,-0.6331331,-0.49353957,0.33942643,-0.16283773,0.06543604,-0.21843964,0.24728857,0.1900347,0.029284768,-0.18502808,-0.45163736,-0.53969014,-0.19316211,-0.10690286,0.1263657,0.55663836,0.3382401,-0.24333411,0.37657914,0.36682534,0.7552188,-0.17325065,0.5889528,-0.14424455,-0.42329037,0.19547813,0.56104326,-0.42245582,0.08086134,0.31203443,-0.02225675,0.26393422,-0.06303409,0.3843684,0.14742017,-0.22651254,-0.22816522,0.23160844,-1.100412,0.3858016,0.27257377,-0.093615055,-0.17177874,-0.21740769,0.23802944,-0.33010513,-0.21266243,-1.0294876,-0.44615027,-0.262009

In [59]:
# query_text = "사무실 인사"
query_text = "대구"
# query_text = "사무실에 갑니다"
# query_text = "사무실"
# query_text = "회사 위치"
# query_text = "회사 인사"

inputs = tokenizer(query_text, padding=True, truncation=True, return_tensors="pt")

embeddings, _ = model(**inputs, return_dict=False)
embedding_arr = embeddings[0][0].detach().numpy()
embedding_str = ",".join(str(x) for x in embedding_arr)
embedding_str = "["+embedding_str+"]"

with pool.connect() as db_conn:
    result = db_conn.execute(
        sqlalchemy.text(
            "SELECT org_text FROM empv4 ORDER BY evec <-> :query_vec LIMIT 5"
        ),
        parameters={"query_vec": embedding_str},
    ).fetchall()

for row in result:
    print(row[0])

부산에 갑니다
서울에 갑니다
인천에 갑니다
사무실이 먼가요
지하철을 탑니다


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import json
with pool.connect() as db_conn:
  db_conn.execute(
    sqlalchemy.text(
      "CREATE EXTENSION IF NOT EXISTS pg_trgm with schema public"
    )
  )
  db_conn.commit()

with pool.connect() as db_conn:
  db_conn.execute(
      sqlalchemy.text(
          """
          CREATE TABLE PROFNLEC(
              info varchar(50) PRIMARY KEY,
              rating float,
              assignment varchar(50),
              team varchar(50),
              grade varchar(50),
              attendance varchar(50),
              test varchar(20),
              origin_text varchar(4096),
              v vector(768)
          )
          """
      )
  )
  db_conn.commit()


In [28]:
file_path = '/content/drive/MyDrive/project.json'
data = json.load(open(file_path, 'r'))

i = 1
with pool.connect() as db_conn:
  for key in data.keys():
    if data[key]['rating'] != 'NULL' :
      rating = data[key]['rating'].split()
      rating = rating[0]
    else : rating = data[key]['rating']
    if data[key]['assignment'] != 'NULL' : assignment = data[key]['assignment']
    else : assignment = data[key]['assignment']
    if data[key]['team_project'] != 'NULL' : team = data[key]['team_project']
    else : team = data[key]['team_project']
    if data[key]['grade'] != 'NULL' : grade = data[key]['grade']
    else : grade = data[key]['grade']
    if data[key]['attendence'] != 'NULL' : attendance = data[key]['attendence']
    else : attendance = data[key]['attendence']
    if data[key]['test_count'] != 'NULL' : test = data[key]['test_count']
    else : test = data[key]['test_count']

    inputs = tokenizer(data[key]['articles'], padding = True, truncation = True, return_tensors = "pt")

    embeddings, _ = model(**inputs, return_dict = False)
    embedding_arr = embeddings[0][0].detach().numpy()
    embedding_str = ",".join(str(x) for x in embedding_arr)
    embedding_str = "["+embedding_str+"]"

    insert_stmt = sqlalchemy.text(
        "INSERT INTO PROFNLEC VALUES (:info, :rating, :assignment, :team, :grade, :attendance, :test, :origin_text, :v)"
    )

    db_conn.execute(
        insert_stmt, parameters={"info": key, "rating": rating, "assignment": assignment, "team":team,
                                 "grade": grade, "attendance": attendance, "test": test,
                                 "origin_text": data[key]['articles'], 'v': embedding_str}
    )
    print("insert %d clear" %i)
    i+=1
    if i == 6 :
      break

  db_conn.commit()

insert 1 clear
insert 2 clear
insert 3 clear
insert 4 clear
insert 5 clear


In [29]:
query_text = "김정근 교수님 어때?"
inputs = tokenizer(query_text, padding=True, truncation=True, return_tensors="pt")

embeddings, _ = model(**inputs, return_dict=False)
embedding_arr = embeddings[0][0].detach().numpy()
embedding_str = ",".join(str(x) for x in embedding_arr)
embedding_str = "["+embedding_str+"]"
print(embedding_str)

with pool.connect() as db_conn:
    result = db_conn.execute(
        sqlalchemy.text(
            "SELECT origin_text FROM PROFNLEC ORDER BY v <-> :query_vec LIMIT 5"
        ),
        parameters={"query_vec": embedding_str},
    ).fetchall()

for row in result:
    print(row[0])

[-0.76962894,-0.2432861,-0.10210554,-0.66779566,0.30106667,0.68757236,-0.060483634,0.14138386,-0.1599881,-0.04035762,-0.2606614,0.11685452,0.1915007,-0.07315703,-0.008578849,0.23003532,0.57874084,0.7391323,-0.7543403,-0.5225227,-0.20232175,0.30652118,0.2043847,-0.5734617,0.12866603,0.19049294,0.47506914,-0.025084745,-0.4840445,-0.3554995,1.2995293,-0.21241832,-0.074016295,0.017604742,0.3858425,-0.4364747,0.13339199,0.13799399,-0.54661214,-0.38259596,-0.8565658,0.29758993,-0.6349559,-0.055210415,-0.17599984,-0.29049745,-0.03718145,0.99002373,-0.35933828,-0.8602017,0.22837669,-0.4035373,-0.3215077,0.4364907,0.35502508,0.77902037,0.07439486,0.092886314,0.7380912,0.19823505,0.060736254,-0.66111255,-0.43244708,-0.26760224,0.4026759,-0.20003405,-0.118852325,0.57931966,-0.29882157,0.2302272,0.046090327,0.42320696,0.138368,-1.4936213,0.27176374,-0.36303788,-0.19522159,0.15734361,0.3936934,0.29871848,-0.020743519,-0.18592636,0.22945154,-0.37421027,0.049621593,-0.11544607,-0.35049883,0.24670993,

In [26]:
with pool.connect() as conn :
  result = conn.execute(
      sqlalchemy.text(
          "select * from PROFNLEC"
      )
  ).fetchall()

for row in result:
  print(row[0])

ResourceClosedError: ignored